In [ ]:
import numpy as np
import cvxpy as cp

from problems.data_generation import generate_problem_data
from problems.problem_definition import BilevelProblem
from algorithms.barrier_blo_box import BarrierBLO
from algorithms.blocc import BLOCC
from algorithms.implicit_gradient_descent import IGD
from algorithms.BiC_GAFFA import BiC_GAFFA
from algorithms.BSG import BSG

n = 60
seed = 42

data = generate_problem_data(n, seed)
problem = BilevelProblem(data)

x_init = np.zeros(n)
y_init = np.zeros(n)
z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
theta_init = np.zeros(n)
y_g_init = np.zeros(n)
y_F_init = np.zeros(n)
mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
hparams = {
        'barrier_blo': {
            'M': 0.001,
            't': 0.01,
            'alpha_x': 0.002,
            'alpha_y': 0.1,
            'beta_y': 1,
            'epsilon_x': 0.1,
            'epsilon_y': 0.01,
            'inner_max_iters': 1000000,
            'outer_max_iters': 100000
        },
        'blocc': {
            'gamma': 100,
            'alpha_x': 0.005,
            'alpha_g_y': 0.01,
            'alpha_F_y': 0.0002,
            'beta_g_y': 0.01,
            'beta_F_y': 0.01,
            'epsilon_x': 0.1,
            'epsilon_inner_y_g': 0.01,
            'epsilon_outer_y_g': 0.01,
            'epsilon_inner_y_F': 0.01,
            'epsilon_outer_y_F': 0.01,
            'maxmin_g_outer_max_iters': 5000,
            'maxmin_F_outer_max_iters': 5000,
            'maxmin_g_inner_max_iters': 5000,
            'maxmin_F_inner_max_iters': 5000,
            'main_max_iters': 5000 
        },
        'bic_gaffa':{
            'alpha':0.001,
            'c':1,
            'tau':1.3,
            'gamma_1':10,
            'gamma_2':1,
            'eta':1,
            'r':1,
            'epsilon':0.1,
            'max_iters':100
        },
        'BSG':{
            'alpha_x':0.001,
            'alpha_y':0.001,
            'alpha_z':0.001,
            'epsilon_x':0.1,
            'epsilon_y':0.1,
            'epsilon_z':0.1,
            'max_iters_x':1000,
            'max_iters_y':1000,
            'max_iters_z':1000
        },
        'IGD': {
            'M': 1e-3,
            't': 1e-3,
            'alpha_x': 0.001,
            'alpha_y': 0.001,
            'epsilon_x': 1e-3,
            'epsilon_y': 1e-3,
            'inner_max_iters': 1000,
            'outer_max_iters': 1000
        }
    }

# BSG Algorithm

In [2]:
bsg_algo = BSG(problem, hparams)

x_opt_bsg, y_opt_bsg, history = bsg_algo.bsg(x_init, y_init, z_init)
print(problem.f(x_opt_bsg, y_opt_bsg))

Inner iter for y=0, Projected Gradient norm w.r.t y=77.81704153080216
Inner iter for y=1, Projected Gradient norm w.r.t y=73.1800657659752
Inner iter for y=2, Projected Gradient norm w.r.t y=68.82482146033996
Inner iter for y=3, Projected Gradient norm w.r.t y=64.73385095526986
Inner iter for y=4, Projected Gradient norm w.r.t y=60.89079941385908
Inner iter for y=5, Projected Gradient norm w.r.t y=57.28034387644168
Inner iter for y=6, Projected Gradient norm w.r.t y=53.88812695587442
Inner iter for y=7, Projected Gradient norm w.r.t y=50.70069486474364
Inner iter for y=8, Projected Gradient norm w.r.t y=47.70543948732987
Inner iter for y=9, Projected Gradient norm w.r.t y=44.89054422843475
Inner iter for y=10, Projected Gradient norm w.r.t y=42.2449333891392
Inner iter for y=11, Projected Gradient norm w.r.t y=39.75822483631061
Inner iter for y=12, Projected Gradient norm w.r.t y=37.42068574828995
Inner iter for y=13, Projected Gradient norm w.r.t y=35.22319123374783
Inner iter for y=1

LinAlgError: Singular matrix

# BiC_GAFFA Feasibility Check

# BiC_GAFFA Algorithm

In [11]:
bic_algo = BiC_GAFFA(problem, hparams)

x_opt_bic, y_opt_bic, history = bic_algo.bic_gaffa(x_init, y_init, z_init, theta_init)
print(problem.f(x_opt_bic, y_opt_bic))

norm of dx, dy, dx: (107.5859948062065, 116.66064647118927, 14086.210449174287)
before: norm of y 0.0
after: norm of y 0.11666064647118926
f(x,y)=-9.995734614174179
norm of dx, dy, dx: (87.2245189232066, 88.89199490249885, 26493.324445047463)
before: norm of y 0.11666064647118926
after: norm of y 0.20500332003848123
f(x,y)=-14.770192419272027
norm of dx, dy, dx: (82.1229564122132, 72.49732444994986, 15061.26212278838)
before: norm of y 0.20500332003848123
after: norm of y 0.276146223413328
f(x,y)=-16.962857324062178
norm of dx, dy, dx: (72.3000163603953, 64.22337845404893, 24706.003865095983)
before: norm of y 0.276146223413328
after: norm of y 0.33803931484118066
f(x,y)=-17.18251879945771
norm of dx, dy, dx: (74.13874357875378, 58.306789180103216, 14421.841307591149)
before: norm of y 0.33803931484118066
after: norm of y 0.39334253730678587
f(x,y)=-16.64566389356304
norm of dx, dy, dx: (67.4496126220538, 55.53159303244553, 24784.56561372405)
before: norm of y 0.39334253730678587
after

# BiC_GAFFA Feasibility Check

In [12]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_bic, y_opt_bic, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_bic, y_opt_bic, i) for i in range(problem.num_constraints_h2)]

y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_bic, y_opt_bic)
print(f"f_theoretical_opt_result={problem.f(x_opt_bic, y_original_opt_blocc)}, f_blocc_numerical_result={problem.f(x_opt_bic, y_opt_bic)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_bic, y_original_opt_blocc)}, g_blocc_numerical_result={problem.g(x_opt_bic, y_opt_bic)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=165.60679282348508, f_blocc_numerical_result=160.52086131580654
g_theoretical_opt_result=-69.16092309568516, g_blocc_numerical_result=-69.17116994160955
h_1[0] = 0.0030596314664246005
h_1[1] = 0.050938476866883936
h_1[2] = -1.1633339886280258
h_1[3] = -0.5257054776068921
h_1[4] = -1.1019034343738923
h_1[5] = -2.35274812406054
h_1[6] = -1.7554510278420952
h_1[7] = 0.007036131739513923
h_1[8] = 0.08186589350713613
h_1[9] = -1.5232631828379128
h_1[10] = -0.08889869456808541
h_1[11] = -1.9679380270136195
h_1[12] = -0.7306886268703393
h_1[13] = -2.4487107463966056
h_1[14] = -0.7048059884743351
h_1[15] = -0.9165171165738606
h_1[16] = -0.04766324249488729
h_1[17] = -0.022693697292843873
h_1[18] = -0.6392417499606275
h_1[19] = -1.8189880160340537
h_2[0] = -1.079456272751977
h_2[1] = -0.9608852187957954
h_2[2] = -0.9877874502646424
h_2[3] = -1.04662404139935
h_2[4] = -1.2762243827477584
h_2[5] = -1.0872592394472087
h_2[6] = -0.9043258753806297
h_2[7] = -0.81230947405514

# BLOCC Algorithm

In [2]:
blocc_algo = BLOCC(problem, hparams)


x_opt_blocc, y_opt_blocc, history = blocc_algo.blocc(x_init, y_g_init, y_F_init, mu_g_init, mu_F_init)
print(problem.f(x_opt_blocc, y_opt_blocc))



Main iteration 1
Inner iter for L_g=0, Projected Gradient norm w.r.t y of L_g=88.00551868170518
Inner iter for L_g=1, Projected Gradient norm w.r.t y of L_g=33.99017230264653
Inner iter for L_g=2, Projected Gradient norm w.r.t y of L_g=13.782442091938574
Inner iter for L_g=3, Projected Gradient norm w.r.t y of L_g=5.825780016678072
Inner iter for L_g=4, Projected Gradient norm w.r.t y of L_g=2.545183960822187
Inner iter for L_g=5, Projected Gradient norm w.r.t y of L_g=1.1405283686353462
Inner iter for L_g=6, Projected Gradient norm w.r.t y of L_g=0.5211737826063966
Inner iter for L_g=7, Projected Gradient norm w.r.t y of L_g=0.24184825230760984
Inner iter for L_g=8, Projected Gradient norm w.r.t y of L_g=0.11363468754129386
Inner iter for L_g=9, Projected Gradient norm w.r.t y of L_g=0.05394553731126194
Inner iter for L_g=10, Projected Gradient norm w.r.t y of L_g=0.025832287403158714
Inner iter for L_g=11, Projected Gradient norm w.r.t y of L_g=0.012461147873186577
Inner loop for L_g

# BLOCC Feasibility Check

In [3]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]

y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_blocc, y_opt_blocc)
print(f"f_theoretical_opt_result={problem.f(x_opt_blocc, y_original_opt_blocc)}, f_blocc_numerical_result={problem.f(x_opt_blocc, y_opt_blocc)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_blocc, y_original_opt_blocc)}, g_blocc_numerical_result={problem.g(x_opt_blocc, y_opt_blocc)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=91.44546599869443, f_blocc_numerical_result=86.32814745801593
g_theoretical_opt_result=-58.737415885149375, g_blocc_numerical_result=-58.712031951601794
h_1[0] = -0.508548209139094
h_1[1] = -0.07280102121837756
h_1[2] = -0.7465781078099742
h_1[3] = -0.42384606639170325
h_1[4] = -0.35773994334311676
h_1[5] = -1.1901883581324673
h_1[6] = 1.1102230246251565e-16
h_1[7] = 8.326672684688674e-17
h_1[8] = -1.526067059527454
h_1[9] = -0.644405212503032
h_1[10] = -0.2248651726826788
h_1[11] = -1.2753899426917719
h_1[12] = -1.5734013747294828
h_1[13] = -0.47582252479212533
h_1[14] = -0.14106459474926453
h_1[15] = 2.220446049250313e-16
h_1[16] = -0.6967112872514934
h_1[17] = -0.5974715658748947
h_1[18] = 1.6653345369377348e-16
h_1[19] = -1.8193131128318991
h_2[0] = -1.101705162524051
h_2[1] = -0.9562403253726748
h_2[2] = -0.8507805239906309
h_2[3] = -1.1196585022183596
h_2[4] = -1.2470990999882907
h_2[5] = -1.1008155252262157
h_2[6] = -0.9236526535089383
h_2[7] = -0.856263

# Implicit Gradient Descent Algorithm

In [4]:
IGD_algo = IGD(problem, hparams)

x_opt_IGD, y_opt_IGD, history = IGD_algo.upper_loop(x_init, y_init)

print("Optimized x:", x_opt_IGD)
print("Optimized y:", y_opt_IGD)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 199
f(x, y) = 115.40705313589443, grad_norm = 81.19200747623061
Outer iteration 2
Inner loop converged at iteration 114
f(x, y) = 110.52305121493208, grad_norm = 71.15075501776145
Outer iteration 3
Inner loop converged at iteration 112
f(x, y) = 106.75748580138787, grad_norm = 62.35776957881825
Outer iteration 4
Inner loop converged at iteration 109
f(x, y) = 103.85427021120798, grad_norm = 54.65740233255515
Outer iteration 5
Inner loop converged at iteration 107
f(x, y) = 101.61429091421482, grad_norm = 47.91314422026023
Outer iteration 6
Inner loop converged at iteration 104
f(x, y) = 99.88482730775011, grad_norm = 42.00567874172534
Outer iteration 7
Inner loop converged at iteration 102
f(x, y) = 98.54834900273016, grad_norm = 36.83060089652421
Outer iteration 8
Inner loop converged at iteration 99
f(x, y) = 97.51472817121459, grad_norm = 32.29665170764032
Outer iteration 9
Inner loop converged at iteration 97
f(x, y) = 96.71446807

# Implicit Gradient Descent Feasibility Check

In [5]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h2)]

y_original_opt_IGD = barrier_algo.Interior_inner_loop(x_opt_IGD, y_opt_IGD)
print(f"f_theoretical_opt_result={problem.f(x_opt_IGD, y_original_opt_IGD)}, f_IGD_numerical_result={problem.f(x_opt_IGD, y_opt_IGD)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_IGD, y_original_opt_IGD)}, g_IGD_numerical_result={problem.g(x_opt_IGD, y_opt_IGD)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=91.37994199200028, f_IGD_numerical_result=93.87773322623164
g_theoretical_opt_result=-58.58527934060348, g_IGD_numerical_result=-59.64760524900554
h_1[0] = -0.5081620763129994
h_1[1] = 0.04333128051506291
h_1[2] = -0.8461271725281907
h_1[3] = -0.43274274292839376
h_1[4] = -0.46670122456347385
h_1[5] = -1.0369825546330562
h_1[6] = 0.2456557311987232
h_1[7] = 0.05974292159849154
h_1[8] = -1.3950454417765186
h_1[9] = -0.7170100440516078
h_1[10] = -0.2847430010688392
h_1[11] = -1.3650553403753674
h_1[12] = -1.4522937238538776
h_1[13] = -0.48074810848580185
h_1[14] = -0.14510918092959313
h_1[15] = 0.1952964266768038
h_1[16] = -0.6634000952778967
h_1[17] = -0.5654655881350267
h_1[18] = 0.8581477824084204
h_1[19] = -1.6676765153564599
h_2[0] = -1.1092309985514617
h_2[1] = -0.9623086953793266
h_2[2] = -0.8868761309502758
h_2[3] = -1.0956195990506705
h_2[4] = -1.2686055634004574
h_2[5] = -1.0857974087286093
h_2[6] = -0.935132006634664
h_2[7] = -0.8281148362869305
h_2[8]

# BFBM Algorithm

In [8]:
barrier_algo = BarrierBLO(problem, hparams)

# y_proj = barrier_algo.project_to_constraints(x_init, y_init)

# h = problem.h_1(x_init, y_proj, 0)

# print(y_proj)

# print(h)



x_opt_barrier, y_opt_barrier, history = barrier_algo.upper_loop(x_init, y_init)

print("Optimized x:", x_opt_barrier)
print("Optimized y:", y_opt_barrier)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 87
f(x, y) = 117.8741735352165, grad_norm of hyperfunction= 87.53359063416562
Outer iteration 2
Inner loop converged at iteration 68
f(x, y) = 129.40765052747867, grad_norm of hyperfunction= 1236.219810032325
Outer iteration 3
Inner loop converged at iteration 85
f(x, y) = 68.82033658493681, grad_norm of hyperfunction= 224.80959777803957
Outer iteration 4
Inner loop converged at iteration 84
f(x, y) = 55.66624331696097, grad_norm of hyperfunction= 210.34242544826122
Outer iteration 5
Inner loop converged at iteration 84
f(x, y) = 45.803879236923656, grad_norm of hyperfunction= 192.55355673613312
Outer iteration 6
Inner loop converged at iteration 83
f(x, y) = 38.44636875242237, grad_norm of hyperfunction= 174.9754572804799
Outer iteration 7
Inner loop converged at iteration 83
f(x, y) = 32.88026935238044, grad_norm of hyperfunction= 159.71558168637884
Outer iteration 8
Inner loop converged at iteration 83
f(x, y) = 28.540931356794655,

In [9]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]

y_original_opt_bfbm = barrier_algo.Interior_inner_loop(x_opt_barrier, y_opt_barrier)
print(f"f_theoretical_opt_result={problem.f(x_opt_barrier, y_original_opt_bfbm)}, f_bfbm_numerical_result={problem.f(x_opt_barrier, y_opt_barrier)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_barrier, y_original_opt_bfbm)}, g_bfbm_numerical_result={problem.g(x_opt_barrier, y_opt_barrier)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=-4.279866301714371, f_bfbm_numerical_result=-3.2051209785721753
g_theoretical_opt_result=-15.3112402591417, g_bfbm_numerical_result=-16.263164044845553
h_1[0] = 0.010463263633866043
h_1[1] = -0.40635199792943744
h_1[2] = -0.3244981071570908
h_1[3] = -0.08051729014573339
h_1[4] = -0.7366023067512114
h_1[5] = -0.4493182571916623
h_1[6] = 0.015919699176890734
h_1[7] = -0.15808990249143515
h_1[8] = -0.7096537612196903
h_1[9] = -0.8920994966535362
h_1[10] = -0.6156392584386917
h_1[11] = -1.0703343446896856
h_1[12] = -0.6833146631854716
h_1[13] = -0.8154464751399882
h_1[14] = -0.42792652827959454
h_1[15] = -0.3141940154561206
h_1[16] = 0.09362054085980491
h_1[17] = -0.9186573503532899
h_1[18] = -0.19812565761652218
h_1[19] = -0.9108372255769338
h_2[0] = -0.9778412564353672
h_2[1] = -0.9686470343469746
h_2[2] = -1.0412602786242866
h_2[3] = -1.0511760612600398
h_2[4] = -1.1763183138573179
h_2[5] = -0.9648679795487325
h_2[6] = -1.1364175140104409
h_2[7] = -1.02423981460

# Plot

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(times, grad_norms, marker='o', markersize=1)
plt.xlabel('CPU Time (s)')
plt.ylabel('Gradient Norm')
plt.title('Gradient Norm vs CPU Time')
plt.yscale('log')
plt.grid(True)
plt.show()


: 